In [2]:
import os, json, re
from typing import Any, Dict, List


In [3]:
# read json file to variable
# with open('../pdf_reader/outputs/mdpi.pdf_processed.json') as f:
with open('../pdf_reader/outputs/mdpi.pdf_processed.json') as f:
    d = json.load(f)
    print(d)

{'schema': 1, 'event': 'ingest', 'source': {'bucket': 'pdf-data', 'object': 'mdpi.pdf', 'etag': '17f1dd801c005a1237c7c42b9a9feada'}, 'metadata': {'title': 'Monitoring Mushroom Growth with Machine Learning', 'authors': 'Vasileios Moysiadis, Georgios Kokkonis, Stamatia Bibi, Ioannis Moscholios, Nikolaos Maropoulos, Panagiotis Sarigiannidis', 'keywords': 'mushroom; YOLOv5; Detectron2; machine learning; object detection; instance segmentation', 'abstract': "Mushrooms contain valuable nutrients, proteins, minerals, and vitamins, and it is suggested to include them in our diet. Many farmers grow mushrooms in restricted environments with specific atmospheric parameters in greenhouses. In addition, recent technologies of the Internet of things intend to give solutions in the agriculture area. In this paper, we evaluate the effectiveness of machine learning for mushroom growth monitoring for the genus Pleurotus. We use YOLOv5 to detect mushrooms' growing stage and indicate those ready to harves

In [4]:
d

{'schema': 1,
 'event': 'ingest',
 'source': {'bucket': 'pdf-data',
  'object': 'mdpi.pdf',
  'etag': '17f1dd801c005a1237c7c42b9a9feada'},
 'metadata': {'title': 'Monitoring Mushroom Growth with Machine Learning',
  'authors': 'Vasileios Moysiadis, Georgios Kokkonis, Stamatia Bibi, Ioannis Moscholios, Nikolaos Maropoulos, Panagiotis Sarigiannidis',
  'keywords': 'mushroom; YOLOv5; Detectron2; machine learning; object detection; instance segmentation',
  'abstract': "Mushrooms contain valuable nutrients, proteins, minerals, and vitamins, and it is suggested to include them in our diet. Many farmers grow mushrooms in restricted environments with specific atmospheric parameters in greenhouses. In addition, recent technologies of the Internet of things intend to give solutions in the agriculture area. In this paper, we evaluate the effectiveness of machine learning for mushroom growth monitoring for the genus Pleurotus. We use YOLOv5 to detect mushrooms' growing stage and indicate those re

In [5]:
text = d["text"]
parts = text.split("===")
parts

['Abstract: Mushrooms contain valuable nutrients, proteins, minerals, and vitamins, and it is suggested to include them in our diet. Many farmers grow mushrooms in restricted environments with speciﬁc atmospheric parameters in greenhouses. In addition, recent technologies of the Internet of things intend to give solutions in the agriculture area. In this paper, we evaluate the effectiveness of machine learning for mushroom growth monitoring for the genus Pleurotus. We use YOLOv5 to detect mushrooms’ growing stage and indicate those ready to harvest. The results show that it can detect mushrooms in the greenhouse with an F1-score of up to 76.5%. The classiﬁcation in the ﬁnal stage of mushroom growth gives an accuracy of up to 70%, which is acceptable considering the complexity of the photos used. In addition, we propose a method for mushroom growth monitoring based on Detectron2. Our method shows that the average growth period of the mushrooms is 5.22 days. Moreover, our method is also 

In [37]:
# Find the index of "introduction" and "references"
introduction_index = next(i for i, section in enumerate(parts) if section == 'introduction')
references_index = next(i for i, section in enumerate(parts) if section == 'references')

# Remove sections before "introduction" and after "references"
parts_with_chapters = parts[introduction_index:references_index]

# Combine the remaining sections with text above 100 characters
parts_without_chapters = [part for part in parts_with_chapters if len(part) > 100]

combined_parts_without_chapters = "\n".join(parts_without_chapters)


In [38]:
parts_with_chapters

['introduction',
 ' Citation: Moysiadis, V.; Kokkonis, G.; Bibi, S.; Moscholios, I.; Maropoulos, N.; Sarigiannidis, P. Monitoring Mushroom Growth with Machine Learning. Agriculture 2023, 13, 223. https://doi.org/10.3390/ agriculture13010223 Copyright: © 2023 by the authors. Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). Mushrooms are a rich source of nutrients, proteins, minerals, and vitamins (B, C, and D) [1]. More than 200 species of edible mushrooms are used as ingredients in traditional foods around the world [2], but only 35 species are cultivated in greenhouses in restricted environments [3]. The mushrooms of the genus Pleurotus are in second place worldwide in the industry. Therefore, improving the production line will have a major impact on the economy of the speciﬁc domain. The authors in [4] presented a co

In [39]:
parts_without_chapters

[' Citation: Moysiadis, V.; Kokkonis, G.; Bibi, S.; Moscholios, I.; Maropoulos, N.; Sarigiannidis, P. Monitoring Mushroom Growth with Machine Learning. Agriculture 2023, 13, 223. https://doi.org/10.3390/ agriculture13010223 Copyright: © 2023 by the authors. Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). Mushrooms are a rich source of nutrients, proteins, minerals, and vitamins (B, C, and D) [1]. More than 200 species of edible mushrooms are used as ingredients in traditional foods around the world [2], but only 35 species are cultivated in greenhouses in restricted environments [3]. The mushrooms of the genus Pleurotus are in second place worldwide in the industry. Therefore, improving the production line will have a major impact on the economy of the speciﬁc domain. The authors in [4] presented a comprehensive revie

In [40]:
combined_parts_without_chapters

' Citation: Moysiadis, V.; Kokkonis, G.; Bibi, S.; Moscholios, I.; Maropoulos, N.; Sarigiannidis, P. Monitoring Mushroom Growth with Machine Learning. Agriculture 2023, 13, 223. https://doi.org/10.3390/ agriculture13010223 Copyright: © 2023 by the authors. Licensee MDPI, Basel, Switzerland. This article is an open access article distributed under the terms and conditions of the Creative Commons Attribution (CC BY) license (https:// creativecommons.org/licenses/by/ 4.0/). Mushrooms are a rich source of nutrients, proteins, minerals, and vitamins (B, C, and D) [1]. More than 200 species of edible mushrooms are used as ingredients in traditional foods around the world [2], but only 35 species are cultivated in greenhouses in restricted environments [3]. The mushrooms of the genus Pleurotus are in second place worldwide in the industry. Therefore, improving the production line will have a major impact on the economy of the speciﬁc domain. The authors in [4] presented a comprehensive review

In [ ]:
def sliding(text: str, chunk_word_size: int = 288, chunk_overlap: int = 0) -> List[Dict[str, Any]]:
    words = text.split(" ")
    out: List[Dict[str, Any]] = []
    for idx, i in enumerate(range(0, len(words), chunk_word_size - chunk_overlap)):
        chunk = words[i:i + chunk_word_size]
        if chunk:
            out.append(
                {
                    "index": idx,
                    "num_words": len(chunk),
                    "text": " ".join(chunk),
                }
            )            
    return out

g = sliding(combined_parts_without_chapters, chunk_word_size=288, chunk_overlap=0)

In [67]:
def create_overlapping_chunks(text, MAX_CHUNK_SIZE=365, OVERLAP_MAX_SIZE=73):
    # List to hold the final output
    out: List[Dict[str, Any]] = []
    # Split the text using ". " and ".<Capital letter>" as delimiters
    pattern = r'(?<=\.)\s+|(?<=\.)(?=[A-Z])'
    sentences = [s for s in re.split(pattern, text) if s]
    # Initialize variables for chunking
    chunks = []
    chunk_words = 0
    for idx,sentence in enumerate(sentences):
        # Split the sentence into words
        words = sentence.split(" ")
        # Start the first chunk with the first sentence
        if chunk_words == 0:
            chunk_sentence_ids = [idx]
            chunk_words = len(words)
        # If the current chunk plus the new sentence is within the max size, add it
        elif chunk_words + len(words) < MAX_CHUNK_SIZE:
            chunk_sentence_ids.append(idx)
            chunk_words += len(words)
        # If adding the new sentence exceeds the max size, create a new chunk
        else:
            chunks.append(chunk_sentence_ids)
            chunk_sentence_ids = []
            overlapping_words = 0
            # Check how many sentences can be added from the end of the current chunk to the new chunk for overlapping
            for y in reversed(chunks[-1]):
                overlapping_words += len(sentences[y].split(" "))
                if overlapping_words < OVERLAP_MAX_SIZE:
                    chunk_sentence_ids.append(y)
                else:
                    break
            chunk_sentence_ids.reverse()
            chunk_sentence_ids.append(idx)
            chunk_words = len(words)

    # Convert from sentence indices to actual text chunks
    text_chunks = []
    for idx, chunk in enumerate(chunks):
        text_chunks.append(" ".join([sentences[i] for i in chunk]))
        out.append(
            {
                "index": idx,
                "num_words": len(text_chunks[-1].split(" ")),
                "text": text_chunks[-1],
            }
        )

    return out


b = create_overlapping_chunks(combined_parts_without_chapters, MAX_CHUNK_SIZE=365, OVERLAP_MAX_SIZE=0)